In [1]:
# 失败的实验，因为KL散度要计算softmax，但是二分类只输出一个值，懒得再去改数据了，直接去做实验得了。

import sys
import os
import numpy as np
import torch
import random

# 添加环境
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../../MyExpr")))
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../../FedML")))
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../../")))

print(sys.path)

['/home/ubuntu/Fed_Expr', '/home/ubuntu/Fed_Expr/FedML', '/home/ubuntu/Fed_Expr/MyExpr', '/home/ubuntu/Fed_Expr/MyExpr/dfl', '/home/ubuntu/miniconda/envs/fedml/lib/python37.zip', '/home/ubuntu/miniconda/envs/fedml/lib/python3.7', '/home/ubuntu/miniconda/envs/fedml/lib/python3.7/lib-dynload', '', '/home/ubuntu/miniconda/envs/fedml/lib/python3.7/site-packages', '/home/ubuntu/miniconda/envs/fedml/lib/python3.7/site-packages/IPython/extensions', '/home/ubuntu/.ipython']


In [2]:
import argparse

# todo 拉配置
parser = argparse.ArgumentParser("cifar")
parser.add_argument('--data_name', type=str, default="SUSY", help='SUSY; RO')
parser.add_argument('--epoch_size', type=int, default=10, help='1,2,3,4,5')
# parser.add_argument('--learning_rate', type=float, default=0.1)
parser.add_argument("--batch_size", type=int, default=100)
parser.add_argument('--weight_decay', type=float, default=0.0001)
parser.add_argument('--beta', type=float, default=0.2, help='beta: 0; 0.2; 0.5')
parser.add_argument('--client_number', type=int, default=100, help='client number')
parser.add_argument('--b_symmetric', type=int, default=0)
parser.add_argument('--topology_neighbors_num_undirected', type=int, default=4)
parser.add_argument('--topology_neighbors_num_directed', type=int, default=4)
parser.add_argument('--input_dim', type=int, default=18)
parser.add_argument('--output_dim', type=int, default=1)
parser.add_argument('--learning_rate', type=float, default=0.01)
parser.add_argument('--latency', type=float, default=0)

# parser.parse_args()
args = parser.parse_known_args()[0]
seed = 1234
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
# args.__setattr__("epoch_size", 100)
# print(args.epoch_size)

In [7]:
# load data
from MyExpr.dfl.data_loader_for_susy_dfl import DataLoader

client_number = args.client_number
client_id_list = [i for i in range(client_number)]
beta = args.beta

reload = False
# 总共 4500000
train_sample_num_in_total = 450000
# todo 测试数据是否划分正确
if reload:
    dataloader = DataLoader("SUSY", "../../FedML/data/UCI/SUSY/SUSY.csv", client_id_list, train_sample_num_in_total, beta)
    batch_data_dict = dataloader.load_data()
    test_X, test_Y = dataloader.test_X, dataloader.test_Y

加载全部数据...完成
加载对抗数据...完成
加载随机数据...完成


In [4]:
# LR model
import torch

# logistic
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        outputs = self.linear(x)
        return outputs

In [43]:
%load_ext autoreload
%autoreload 2

from FedML.fedml_api.standalone.decentralized.topology_manager import TopologyManager
from MyExpr.dfl.client_test import ClientTEST

# train
b_symmetric = args.b_symmetric
topology_neighbors_num_undirected = args.topology_neighbors_num_undirected
topology_neighbors_num_directed = args.topology_neighbors_num_directed
input_dim = args.input_dim
output_dim = 2

iteration = int(train_sample_num_in_total / (client_number * args.batch_size))

# print(train_sample_num_in_total / (client_number * args.batch_size))

epoch = args.epoch_size

if b_symmetric:
    topology_manager = TopologyManager(client_number, True,
                                       undirected_neighbor_num=topology_neighbors_num_undirected)
else:
    topology_manager = TopologyManager(client_number, False,
                                       undirected_neighbor_num=topology_neighbors_num_undirected,
                                       out_directed_neighbor=topology_neighbors_num_directed)
topology_manager.generate_topology()
print("finished topology generation")

client_list = []
# todo 搞清楚latency
for client_id in client_id_list:
    model = LogisticRegression(input_dim, output_dim)
    client_data = batch_data_dict[client_id]
    client = ClientTEST(model, client_id, client_data, topology_manager, iteration, learning_rate=args.learning_rate, batch_size=args.batch_size, weight_decay=args.weight_decay, latency=args.latency, b_symmetric=b_symmetric)
    client_list.append(client)
print("finished client generation")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
finished topology generation
finished client generation


In [44]:
import logging

test = True

# 100(client_number)
# 45(iteration) * 100(batch_size)
# print(len(dataloader.BatchDataDict[1]['x'][0]))

# check是不是每个都是相同的
# for client in client_list:
#     size = 100
#     for i in range(len(client.streaming_data['x'])):
#         if len(client.streaming_data['x'][i]) != size:
#             print(client.id, i)



for e in range(epoch):
    # train
    regret = 0
    correct = 0
    total = 0
    for t in range(iteration):
        # logging.info('--- Iteration %d ---' % t)
        # print('--- Iteration %d ---' % t)
        # 本地训练 -> 广播
        for client in client_list:
            # print(type(client.streaming_data))
            # print(len(client.streaming_data['x']))
            # print(len(client.streaming_data['x'][t]), type(client.streaming_data['x'][t]), type(np.asarray(client.streaming_data['x'][t], dtype=np.float32)))
            
            client.train(t)
            client.send_local_gradient_to_neighbor(client_list)

        # Mutual Learning
        for client in client_list:
            top_k = client.top_k_by_loss(client_list, t)
            client.mutual_update(top_k)
            regret += client.get_regret()[t]
            correct += client.get_record()[t][0]
            total += client.get_record()[t][1]

    print(client.loss_in_each_iteration)
    print("epoch:", e, "train_regret:", regret, "train_acc:", float(correct / train_sample_num_in_total), "epoch:", e)

    # test (如果说是用个性化模型，那应该不能用统一的test集去检验)
    regret = 0
    correct = 0
    with torch.no_grad():
        for client in client_list:
            model = client.model
            for i in range(len(test_X)):
                x, y = torch.from_numpy(np.asarray(test_X[i], dtype=np.float32)).float(), torch.from_numpy(np.asarray(test_Y[i], dtype=np.float32))
                outputs = model(x)
                
                loss = client.criterion(outputs.squeeze(), y)
                pred = outputs.ge(0.5).float()
                correct += (pred.squeeze(1) == y).sum().item()
                regret += loss.detach().numpy()
        print("epoch:", e, "test_regret:", regret, "test_acc:", float(correct / (len(test_X) * len(client_list) * args.batch_size)))

    if test:
        break

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)